In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import librosa
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
from algorithms import label_encode_columns, svm_model, accuracy_calculator, random_forest_model, array_column_spread, one_hot_encode_columns
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv("dsl_data/development.csv")
df_eval = pd.read_csv("dsl_data/evaluation.csv")

In [3]:
df['intent'] = df['action']+ df['object']

In [4]:
# y encoded done
dict = {}

for i, el in enumerate(df['intent'].unique()):
    dict[el] = i

df['intent'] = df['intent'].apply(lambda x: dict[x])

In [ ]:
columns = ["Self-reported fluency level ","First Language spoken", "Current language used for work/school", "gender"]

for column in columns:
    encoder = OneHotEncoder()
    encoded_data = encoder.fit_transform(df[[column]])
    feature_names = encoder.get_feature_names_out([column])
    encoded_df = pd.DataFrame(encoded_data.toarray(), columns=feature_names)
    df.drop(column, axis=1, inplace=True)
    df = pd.concat([df, encoded_df], axis=1)

In [ ]:
for column in columns:
    encoder = OneHotEncoder()
    encoded_data = encoder.fit_transform(df_eval[[column]])
    feature_names = encoder.get_feature_names_out([column])
    encoded_df = pd.DataFrame(encoded_data.toarray(), columns=feature_names)
    df_eval.drop(column, axis=1, inplace=True)
    df_eval = pd.concat([df_eval, encoded_df], axis=1)

In [ ]:
le = LabelEncoder()
label = le.fit_transform(df['ageRange'])
df.drop('ageRange', axis=1, inplace=True)
df['ageRange'] = label

In [ ]:
le = LabelEncoder()
label = le.fit_transform(df_eval['ageRange'])
df_eval.drop('ageRange', axis=1, inplace=True)
df_eval['ageRange'] = label

In [ ]:

audio_feature_extraction(df)
audio_feature_extraction(df_eval)

In [ ]:
from scipy.stats import skew, kurtosis

def time_domain(df, column):
    df[f'{column}_mean'] = df[column].apply(lambda x: np.mean(x,axis=1))
    df[f'{column}_min'] = df[column].apply(lambda x: np.min(x,axis=1))
    df[f'{column}_max'] = df[column].apply(lambda x: np.max(x,axis=1))
    df[f'{column}_skew'] = df[column].apply(lambda x: skew(x,axis=1))
    df[f'{column}_kurtosis'] = df[column].apply(lambda x: kurtosis(x,axis=1))
    df[f'{column}_std'] = df[column].apply(lambda x: np.std(x,axis=1))

In [ ]:
# down to 4 features
  


In [ ]:
time_domain2(df,'sf')
time_domain2(df_eval,'sf')

In [ ]:

sro_feature(df)
sro_feature(df_eval)

In [ ]:
time_domain2(df,'spectral_rolloff')
time_domain2(df_eval,'spectral_rolloff')

In [ ]:

zcr_feature(df)
zcr_feature(df_eval)

In [ ]:

mfcc_feature(df)
mfcc_feature(df_eval)

In [ ]:
X = df2.copy()
X2 = df3.copy()

In [ ]:
X.drop(columns=['Id', 'path', 'speakerId', 'action', 'object', 'intent'], inplace=True)

X.to_csv('x.csv',index=False)

X2.drop(columns=['Id', 'path', 'speakerId'], inplace=True)


In [ ]:
import matplotlib.pyplot as plt



In [ ]:
# from sklearn.decomposition import PCA
# import pandas as pd

# # Assuming your dataframe is named 'df'

# # Create an instance of PCA
# pca = PCA()

# # Fit PCA on the data
# pca.fit(X)

# # Get the explained variance ratio of each principal component
# var_ratio = pca.explained_variance_ratio_

# # Get the cumulative explained variance ratio
# cum_var_ratio = np.cumsum(var_ratio)

# # Plot the explained variance ratio
# plt.bar(range(1, len(var_ratio)+1), var_ratio, alpha=0.5, align='center', label='individual explained variance')
# plt.step(range(1, len(cum_var_ratio)+1), cum_var_ratio, where='mid', label='cumulative explained variance')
# plt.ylabel('Explained variance ratio')
# plt.xlabel('Principal component index')
# plt.legend(loc='best')
# plt.show()


# # Create an instance of PCA, specifying the number of components to keep
# pca = PCA(n_components=75)

# # Fit PCA on the data
# df_pca = pca.fit_transform(X)


In [10]:
y = df['intent'].values
X = pd.read_csv("x.csv")
X_eval = pd.read_csv("x_eval.csv")

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def normalize_dataframe(df):
    scaler = MinMaxScaler()
    df[df.columns] = scaler.fit_transform(df[df.columns])
    return df

X =  normalize_dataframe(X)
X_eval = normalize_dataframe(X_eval)

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
hyperparams = [('linear', 0.1, 0.01), ('linear', 0.1, 0.1), ('linear', 0.1, 1), ('linear', 1, 0.01), ('linear', 1, 0.1), ('linear', 1, 1), ('linear', 10, 0.01), ('linear', 10, 0.1), ('linear', 10, 1), ('rbf', 0.1, 0.01), ('rbf', 0.1, 0.1), ('rbf', 0.1, 1), ('rbf', 1, 0.01), ('rbf', 1, 0.1), ('rbf', 1, 1), ('rbf', 10, 0.01), ('rbf', 10, 0.1), ('rbf', 10, 1)]
def svm_model2(X_train, y_train, X_test, hyperparams):
    clf = SVC(C = hyperparams[1], gamma = hyperparams[2], kernel = hyperparams[0])
    # train the model on the training data
    clf.fit(X_train, y_train)
    # predict the target values for the test data
    # returning the y_predict
    return clf.predict(X_test)
for i in hyperparams:
    y_pred = svm_model2(X_train, y_train, X_test,i)
    print(accuracy_calculator(y_test, y_pred),i)


0.519533231861999 ('linear', 1, 0.1)
0.519533231861999 ('linear', 1, 1)
0.5256215119228818 ('linear', 10, 0.01)


In [ ]:
y_pred = random_forest_model(X_train, X_test, y_train)
random_forest_accuracy = accuracy_calculator(y_test, y_pred)
random_forest_accuracy

In [ ]:
clf = RandomForestClassifier(n_estimators=100, random_state=0)

# shuffle the data before performing k-fold cross validation
X_shuffled, y_shuffled = shuffle(X, y)

# perform k-fold cross validation with 5 folds
scores = cross_val_score(clf, X_shuffled, y_shuffled, cv=10)

# calculate the mean accuracy of the model across all folds
accuracy = np.mean(scores)
print("Accuracy:", accuracy)


In [ ]:
from sklearn.svm import SVC


# def svm_model2(X_train, y_train, X_test):
#     clf = SVC()
#     # train the model on the training data
#     clf.fit(X_train, y_train)
#     # predict the target values for the test data
#     # returning the y_predict
#     return clf.predict(X_test)
# y_pred = svm_model2(X_train, y_train, X_test)
# svm_accuracy = accuracy_calculator(y_test, y_pred)

In [ ]:
# svm_accuracy

In [ ]:
# clf = RandomForestClassifier(n_estimators=100, random_state=0)

# # shuffle the data before performing k-fold cross validation
# X_shuffled, y_shuffled = shuffle(X, y)

# # perform k-fold cross validation with 5 folds
# scores = cross_val_score(clf, X_shuffled, y_shuffled, cv=5)

# # calculate the mean accuracy of the model across all folds
# accuracy = np.mean(scores)
# print("Accuracy:", accuracy)


In [ ]:
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the training data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Initialize PCA
pca = PCA()

# Fit PCA to the scaled training data
X_train_pca = pca.fit_transform(X_train_scaled)

# Get the explained variance ratio
explained_variance = pca.explained_variance_ratio_

# Calculate the cumulative explained variance
cumulative_explained_variance = np.cumsum(explained_variance)

# # Print the explained variance and cumulative explained variance
# print("Explained Variance: ", explained_variance)
# print("Cumulative Explained Variance: ", cumulative_explained_variance)

# Determine the number of components to keep
n_components = np.argmax(cumulative_explained_variance >= 0.9) + 1

# Re-initialize PCA with the number of components to keep
pca = PCA(n_components=n_components)

# Fit PCA to the scaled training data
X_train_pca = pca.fit_transform(X_train_scaled)

# Get the indices of the top n features
feature_indices = np.argmax(pca.components_, axis=1)

# Print out the top n features
top_features = [X.columns[i] for i in feature_indices]

# Transform the test set
X_test_scaled = scaler.transform(X_test)
X_test_pca = pca.transform(X_test_scaled)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.svm import SVC


# Create a new SVM model
svm = SVC()

# Use grid search to find the optimal parameters
param_grid = {'kernel': ['linear', 'rbf'], 'C': [0.1, 1, 10], 'gamma': [0.01, 0.1, 1]}
grid_search = GridSearchCV(svm, param_grid, cv=10)
grid_search.fit(X_train_pca, y_train)

# Print out the best parameters
print("Best Parameters: ", grid_search.best_params_)


# evaluate the best model on the test data
best_model = grid_search.best_estimator_
best_score = best_model.score(X_test_pca, y_test)
print("Best Score on Test Data: ", best_score)

In [ ]:
# svm_model = SVC(C = 10, gamma = 0.1, kernel = 'rbf')
# svm_model.fit(X_train_pca, y_train)

# # evaluate the model with test data
# test_score = svm_model.score(X_test_pca, y_test)
# print("Test Score: ", test_score)

for i in hyperparams:
    y_pred = svm_model2(X_train_pca, y_train, X_test_pca,i)
    print(accuracy_calculator(y_test, y_pred),i)

In [ ]:
X_test_pca = normalize_dataframe(pd.DataFrame(X_test_pca))

In [ ]:
X_test_pca

In [ ]:
# Step 4: Select top n features
from sklearn.feature_selection import SelectKBest, f_classif
selector = SelectKBest(f_classif, k=80)
selector.fit(X_train_pca, y_train)
X_train_best = selector.transform(X_train_pca)
X_test_best = selector.transform(X_test_pca)
# Get the indices of the top n features
feature_indices = selector.get_support(indices=True)
best_n_features = [X.columns[i] for i in feature_indices]


In [13]:
svm_model = SVC(C = 10, gamma = 0.01, kernel = 'rbf')
svm_model.fit(X_train_pca, y_train)

NameError: name 'X_train_pca' is not defined

In [16]:
np.unique(y_pred)

array([3])

In [ ]:
svm_accuracy

In [ ]:
# from sklearn.metrics import make_scorer, accuracy_score
# from sklearn.model_selection import GridSearchCV
# from sklearn.feature_selection import RFE


# # Initialize the feature matrix and target variable

# # Initialize the SVM model
# svm = SVC(kernel="linear")

# # Initialize RFE 
# rfe = RFE(svm)

# # Define the grid of values for the number of features to select and the accuracy required
# param_grid = {'n_features_to_select':[60,70,80], 
#               'estimator__C':[0.1, 1, 10], 
#               'estimator__kernel':['linear', 'rbf', 'poly'], 
#               'estimator__gamma': [0.1, 1, 10]}

# # Define the scoring function
# acc_scorer = make_scorer(accuracy_score)

# # Initialize GridSearchCV
# grid_search = GridSearchCV(rfe, param_grid, scoring=acc_scorer)

# # Fit the grid_search to the data
# grid_search.fit(X_train, y_train)

# best_params = grid_search.best_params_
# best_score = grid_search.best_score_
# support = grid_search.best_estimator_.support_

# # Get the feature names
# feature_names = X.columns

# # Get the best features
# best_features = feature_names[support]

# # Print the best parameters, best score, and best features
# print("Best parameters: ", best_params)
# print("Best score: ", best_score)
# print("Best features: ", best_features)